# 프롬프트 엔지니어링 기법
https://www.promptingguide.ai/kr/techniques

In [15]:
from dotenv import load_dotenv
import os

# .env 파일의 내용을 환경 변수로 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

### Zero-shot Prompting
: 예시가 주어지지 않음

In [39]:
prompt = '''
다음 영어 문장을 불어로 번역해줘 :
'Hello, how are you?'
'''
completion = client.chat.completions.create(
    model = "gpt-4o-mini",
    temperature = 0.2,
    messages = [
        { "role" : "user",
          "content" : prompt }
    ]
)

print(completion.choices[0].message.content)

'Hello, how are you?'는 프랑스어로 'Bonjour, comment ça va ?'입니다.


### Few-shot Prompting 
: 몇 개의 예시를 제공 , 한 개의 예시가 주어지는 경우 --> One-shot

In [44]:
prompt = '''
English : 'Hello, how are you?'
French : 'Bonjour, comment ça va ?'

English: 'What is your name?'
French: 'Comment vous appelez-vous?'

English : 'Hello, Miss'
French :
'''
completion = client.chat.completions.create(
    model = "gpt-4o-mini",
    temperature = 0.2,
    messages = [
        { "role" : "user",
          "content" : prompt }
    ]
)

print(completion.choices[0].message.content)
# 지시가 없어도 예시만 보고 불어로 번역해준다

French: 'Bonjour, Mademoiselle'


In [56]:
prompt = """
아래 예시를 참조해서 알맞은 답변해줘

질문: 이 영화 너무 재미 없어!!'
답변: 부정 평가

질문: 이 영화 돈이 안 아까와..'
답변: 긍정 평가

질문: 시간만 버렸어~'
답변: 부정 평가

질문: 내내 하품만 나옴'
답변: 

질문: 핵 꿀잼~'
답변: 
"""
completion = client.chat.completions.create(
    model = "gpt-4o-mini",
    temperature = 0.2,
    messages = [
        { "role" : "user",
          "content" : prompt }
    ]
)

print(completion.choices[0].message.content)

질문: 내내 하품만 나옴  
답변: 부정 평가

질문: 핵 꿀잼~  
답변: 긍정 평가


### CoT(Chain-of-Thought) Prompting

In [78]:
prompt = """
철수에게는 사과가 10개 있었습니다 그저께 부터 오늘까지 매일 아침에 2개씩 먹었습니다
오늘 점심에 어머니께서 사과 5개를 더 사다 주셨습니다 오늘 저녁에 친구에게 2개를 주었습니다
철수는 총 몇 개의 사과를 가지고 있을까요?
"""
completion = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    temperature = 0,
    messages = [
        { "role" : "user",
          "content" : prompt }
    ]
)

print(completion.choices[0].message.content)  # 11개,9개,5개 .. : 틀린 답변이 자주 나옴

철수는 현재 11개의 사과를 가지고 있습니다. 처음에는 10개의 사과를 가졌고, 그저께와 어제 각각 2개씩 먹어서 6개가 줄었으며, 어머니께서 5개를 더 사다 주셨기 때문에 11개가 되었습니다. 이후 친구에게 2개를 주었기 때문에 현재 11개의 사과를 가지고 있습니다.


In [70]:
completion = client.chat.completions.create(
    model = "gpt-4o-mini",
    temperature = 0,
    messages = [
        { "role" : "user",
          "content" : prompt }
    ]
)

print(completion.choices[0].message.content)   # 7개 : 정답

철수의 사과 개수를 계산해 보겠습니다.

1. 철수는 처음에 사과가 10개 있었습니다.
2. 그저께부터 오늘까지 매일 아침에 2개씩 먹었습니다. 오늘은 3일째이므로:
   - 2개 × 3일 = 6개를 먹었습니다.
3. 따라서 오늘 아침까지 남은 사과는:
   - 10개 - 6개 = 4개입니다.
4. 오늘 점심에 어머니께서 사과 5개를 더 사다 주셨으므로:
   - 4개 + 5개 = 9개입니다.
5. 오늘 저녁에 친구에게 2개를 주었으므로:
   - 9개 - 2개 = 7개입니다.

결론적으로, 철수는 오늘 저녁에 총 7개의 사과를 가지고 있습니다.


In [74]:
prompt = """
철수에게는 사과가 10개 있었습니다 그저께 부터 오늘까지 매일 아침에 2개씩 먹었습니다
오늘 점심에 어머니께서 사과 5개를 더 사다 주셨습니다 오늘 저녁에 친구에게 2개를 주었습니다
철수는 총 몇 개의 사과를 가지고 있을까요? 단계별로 분리해서 생각해 보세요
"""
completion = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    temperature = 0,
    messages = [
        { "role" : "user",
          "content" : prompt }
    ]
)

print(completion.choices[0].message.content)

1. 처음에 가지고 있던 사과의 개수: 10개
2. 매일 아침에 먹은 사과의 개수: 2개 * 3일 = 6개
3. 어머니가 사다준 사과의 개수: 5개
4. 친구에게 준 사과의 개수: 2개

따라서, 철수가 현재 가지고 있는 사과의 개수는
10개 (처음에 가지고 있던 개수) - 6개 (아침에 먹은 개수) + 5개 (어머니가 사다준 개수) - 2개 (친구에게 준 개수) = 7개

철수는 현재 7개의 사과를 가지고 있습니다.


In [80]:
prompt = """
1. 철수에게는 사과가 10개 있었습니다.
2. 그저께부터 오늘까지 매일 아침에 2개씩 먹었습니다.
   (3일 동안 2개씩 먹어서 2 * 3 = 6개의 사과를 먹었습니다.)
3. 오늘 점심에 어머니께서 사과 5개를 더 사다 주셨습니다.
4. 오늘 저녁에 친구에게 2개를 주었습니다.
철수는 총 몇 개의 사과를 가지고 있을까요?
"""
completion = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    temperature = 0,
    messages = [
        { "role" : "user",
          "content" : prompt }
    ]
)

print(completion.choices[0].message.content) # 7개 : 정답

철수는 현재 7개의 사과를 가지고 있습니다. 처음에 10개의 사과가 있었고, 6개를 먹었으며, 5개를 추가로 받았지만 2개를 친구에게 주었기 때문에 7개가 남게 됩니다.
